# CMS Dimuon spectrum analyis in Python with an explicit event loop

Analysis running time: 4h 54mn (4h 53mn CPU) on my Laptop (i7-8550U CPU @ 1.80GHz, 4 cores, 8 GByte RAM, with SSD).


# Analysis of the di-muon spectrum using data from the CMS detector

This analysis takes data from the CMS experiment recorded in 2012 during Run B and C and extracts the di-muon spectrum. The di-muon spectrum is computed from the data by calculating the invariant mass of muon pairs with opposite charge. In the resulting plot, you are able to rediscover particle resonances in a wide energy range from the [eta meson](https://en.wikipedia.org/wiki/Eta_meson) at about 548 MeV up to the [Z boson](https://en.wikipedia.org/wiki/W_and_Z_bosons) at about 91 GeV.

The analysis code opens an interactive plot, which allows to zoom and navigate in the spectrum. Note that the bump at 30 GeV is not a resonance but an effect of the data taking due to the used trigger. The technical description of the dataset can be found in the respective record linked below.

The result of this analysis can be compared with [an official result of the CMS collaboration using data taken in 2010](https://cds.cern.ch/record/1456510), see the plot below:

![](http://cds.cern.ch/record/1456510/files/pictures_samples_dimuonSpectrum_40pb-1_mod-combined.png)

# Dataset description

The dataset consists of the following columns.

| Column name | Data type | Description |
|-------------|-----------|-------------|
| `nMuon` | `unsigned int` | Number of muons in this event |
| `Muon_pt` | `float[nMuon]` | Transverse momentum of the muons (stored as an array of size `nMuon`) |
| `Muon_eta` | `float[nMuon]` | Pseudorapidity of the muons |
| `Muon_phi` | `float[nMuon]` | Azimuth of the muons |
| `Muon_mass` | `float[nMuon]` | Mass of the muons |
| `Muon_charge` | `int[nMuon]` | Charge of the muons (either 1 or -1) |

# Data analysis

The following part of the notebook performs the data analysis of the dataset.

We will use ROOT.

In [4]:
import ROOT as R

## Input NanoAOD file

In [3]:
#fname = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
fname = "Run2012BC_DoubleMuParked_Muons.root"

In [3]:
f = R.TFile(fname)

## Function to select event

In [4]:
def select(event):
    return     event.nMuon == 2 \
           and event.Muon_charge[0] != event.Muon_charge[1]

## Function to compute the dilepton mass

In [5]:
def dimu_mass(event):
    p = [ R.Math.PtEtaPhiMVector(event.Muon_pt[i], event.Muon_eta[i], 
                                 event.Muon_phi[i], event.Muon_mass[i]) for i in range(2) ]
    return (p[0] + p[1]).M()
         

## Book the histogram to produce

In [6]:
bins = 30000 # Number of bins in the histogram
low = 0.25 # Lower edge of the histogram
up = 300.0 # Upper edge of the histogram
hmass = R.TH1D("hmass", "Dimuon mass", bins, low, up)

## Run the analysis: the event loop

In [7]:
%%time
tree = f.Events
for e in tree:
    if select(e):
        hmass.Fill(dimu_mass(e))

CPU times: user 4h 4min 42s, sys: 45.5 s, total: 4h 5min 27s
Wall time: 4h 5min 18s


CPU times: user 4h 53min 40s, sys: 50.9 s, total: 4h 54min 31s
Wall time: 4h 54min 20s

## Plot the result

In [14]:
R.gStyle.SetOptStat(0)
R.gStyle.SetTextFont(42)
c = R.TCanvas("c", "", 800, 700)
c.SetLogx()
c.SetLogy();
hmass.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [15]:
label = R.TLatex()
label.SetTextAlign(22)
label.DrawLatex(0.55, 3.0e4, "#eta")
label.DrawLatex(0.77, 7.0e4, "#rho,#omega")
label.DrawLatex(1.20, 4.0e4, "#phi")
label.DrawLatex(4.40, 1.0e5, "J/#psi")
label.DrawLatex(4.60, 1.0e4, "#psi'")
label.DrawLatex(12.0, 2.0e4, "Y(1,2,3S)")
label.DrawLatex(91.0, 1.5e4, "Z")
label.SetNDC(True)
label.SetTextAlign(11)
label.SetTextSize(0.04)
label.DrawLatex(0.10, 0.92, "#bf{CMS Open Data}")
label.SetTextAlign(31)
label.DrawLatex(0.90, 0.92, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}");

In [16]:
%jsroot on
c.Draw()